## Kautilya Assignment 3

In [1]:
!pip install selenium

You should consider upgrading via the 'C:\Users\Kautilya\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import selenium
import re
import time

In [3]:
from selenium.webdriver import Chrome, Firefox
from selenium.webdriver.support.ui import Select
from selenium import webdriver

In [4]:
# subject to change to your own driver
# browser = Firefox()
browser = webdriver.Firefox()
url='https://appsrv.pace.edu/ScheduleExplorerLive/'
browser.get(url)

KeyboardInterrupt: 

## Interaction 

In [5]:
term=Select(browser.find_element_by_id('checkterm'))
term.select_by_visible_text('Fall 2020')
time.sleep(1)

In [6]:
level=Select(browser.find_element_by_id('level'))
level.select_by_visible_text('Graduate')
time.sleep(1)

In [7]:
subject=Select(browser.find_element_by_id('subject'))
subjects= [subject.text for subject in subject.options]
time.sleep(1)

In [8]:
IS_text=[subject for subject in subjects if 'IS' in subject][0]
subject.select_by_visible_text(IS_text)
search_button=browser.find_element_by_id('submitbutton')
search_button.click()

In [9]:
time.sleep(3)
## Increase the table entries
display_entries=Select(browser.find_element_by_name('mainResultsTable_length'))
display_entries.select_by_value('100')

### 1. Extract the list from the page

In [10]:
html=browser.page_source
courses=pd.read_html(html)[0]
soup=BeautifulSoup(html,'html.parser')
links=soup.find_all('a',{'href':True})

## Links for CRN numbers
links_selected=soup.find_all('a',{'data-toggle':'modal'})
crn_links_array=[link for link in links_selected  if link.next.strip().isnumeric()]

### 2. Browse the links 
function to return one pop up message

In [11]:
def read_crn_html(crn_link):
    link=browser.find_element_by_partial_link_text(crn_link.text.strip())
    time.sleep(1)
    link.click()
    popup_view=browser.find_element_by_id('modalBodyContentDiv')
    
    df=pd.read_html(popup_view.get_attribute('innerHTML'))[0].T
    df=pd.read_html(popup_view.get_attribute('innerHTML'))[0].T
    header=df.iloc[0]
    df=df[1:]
    df.columns=header
   
    popup_head=browser.find_element_by_class_name('modal-header')
    header_close_button=popup_head.find_element_by_tag_name('Button')
    time.sleep(2)
    header_close_button.click()
    return df

In [12]:
dfs=[]
for crn_link in crn_links_array:
    try:
        df=read_crn_html(crn_link)
        dfs.append(df)
    except Exception:
        continue

In [13]:
all_course_desc=pd.concat(dfs)

In [14]:
all_course_desc.head()

,COURSE TITLE:,SUBJECT/COURSE#:,CRN:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
1,Advanced Spreadsheet Skills,IS 522,73592,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,Introduction to Coding,IS 612,70426,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
1,Database Management Systems,IS 613,70376,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,71721,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,72779,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...


## Combine datasets 

In [15]:
all_course_desc['CRN:']=pd.to_numeric(all_course_desc['CRN:'])

In [16]:
main_table=pd.read_html(html)[0]

In [17]:
main_table.head()

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,Time,Capacity,Seats Avail.,Instructor,More Info
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,NaN,25,18,Henry Gaylord,More Info
1,70426,IS,612,Introduction to Coding,LEC,3,NYC,Web-assisted (WA),NaN,W,06:10pm-09:00pm,23,03,Dhruvil Gandhi,More Info
2,70376,IS,613,Database Management Systems,LEC,3,NYC,Web-assisted (WA),NaN,T,06:10pm-09:00pm,30,CLOSED,Nathifa Lewis,Prerequisites
3,71721,IS,613,Database Management Systems,LEC,3,Online,Synchronous (SYNC),NaN,T,06:10pm-09:00pm,30,01,Helen Uzamere,Prerequisites
4,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,09:00am-12:00pm,30,14,Paul Dantzig,Prerequisites


In [18]:
all_course_desc=all_course_desc.rename({'CRN:':'CRN'}, axis='columns')
all_course_desc.head()

,COURSE TITLE:,SUBJECT/COURSE#:,CRN,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
1,Advanced Spreadsheet Skills,IS 522,73592,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,Introduction to Coding,IS 612,70426,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
1,Database Management Systems,IS 613,70376,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,71721,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,72779,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...


In [19]:
combined_IS=main_table.merge(all_course_desc, how='left', on='CRN')
combined_IS.columns

Index(['CRN', 'Subject', 'Course#', 'Title', 'Sched Type', 'Credits', 'Campus',
       'Instructional Method*', 'Section Comments', 'Days**', 'Time',
       'Capacity', 'Seats Avail.', 'Instructor', 'More Info', 'COURSE TITLE:',
       'SUBJECT/COURSE#:', 'CREDIT RANGE:', 'SCHOOL:', 'DEPARTMENT:', 'LEVEL:',
       'COREQUISITE:', 'PREREQUISITE:', 'DESCRIPTION:'],
      dtype='object')

In [20]:
combined_IS

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,...,More Info,COURSE TITLE:,SUBJECT/COURSE#:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Advanced Spreadsheet Skills,IS 522,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,70426,IS,612,Introduction to Coding,LEC,3,NYC,Web-assisted (WA),NaN,W,...,More Info,Introduction to Coding,IS 612,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
2,70376,IS,613,Database Management Systems,LEC,3,NYC,Web-assisted (WA),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
3,71721,IS,613,Database Management Systems,LEC,3,Online,Synchronous (SYNC),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
4,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
5,70063,IS,617,Information Systems Principles,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Information Systems Principles,IS 617,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course examines manag...
6,71371,IS,620,IS and Organizational Strategy,LEC,3,NYC,HyFlex (HYFLX),NaN,W,...,More Info,Information Systems and Organizational Strategy,IS 620,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course brings to life...
7,70377,IS,623,Info Sys Design & Devlpmt,LEC,3,NYC,HyFlex (HYFLX),NaN,M,...,More Info,Information Systems Design and Development,IS 623,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Not open to students who have taken IS 621. Co...
8,71427,IS,623,Info Sytems Design & Devlpmt,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Information Systems Design and Development,IS 623,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Not open to students who have taken IS 621. Co...
9,72669,IS,628,Research Mthds for User Exp.,LEC,3,Online,Mixed (MIXAS),"Meeting 163 William St, Rm 238",W,...,More Info,Research Methods for User Experience,IS 628,3.000,Seidenberg School of CSIS,Information Systems,Graduate,None,None,"Course Description: In User Experience, the fo..."


In [21]:
def clean_description(text,word='Course Description:'):
    if isinstance(text,str) and text.startswith(word):
        index = text.index(word) + len(word)
        return text[index:]
    else:
        return text

In [22]:
 combined_IS['DESCRIPTION:'] = combined_IS['DESCRIPTION:'].apply(clean_description)

In [23]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kautilya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kautilya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

def clean(text):
    ### Remove non-space alpha-num character
    text=re.sub(r'[^\w\s]',' ',text).lower()   
    ### Split with non-alphanumeric character 
    words=re.split(r'\W+', text)
    ### Remove stop words            
    non_stop_words=[word for word in words if word not in stopword]        
    ### Clean text
    lem_words= [wn.lemmatize(word) for word in non_stop_words]            
    clean_text= ' '.join(lem_words)
    return clean_text

In [25]:
combined_IS.columns

Index(['CRN', 'Subject', 'Course#', 'Title', 'Sched Type', 'Credits', 'Campus',
       'Instructional Method*', 'Section Comments', 'Days**', 'Time',
       'Capacity', 'Seats Avail.', 'Instructor', 'More Info', 'COURSE TITLE:',
       'SUBJECT/COURSE#:', 'CREDIT RANGE:', 'SCHOOL:', 'DEPARTMENT:', 'LEVEL:',
       'COREQUISITE:', 'PREREQUISITE:', 'DESCRIPTION:'],
      dtype='object')

In [26]:
df_IS = combined_IS[['CRN', 'Subject', 'Course#', 'Title','DESCRIPTION:']]

In [27]:
df_IS = df_IS.dropna()
df_IS

,CRN,Subject,Course#,Title,DESCRIPTION:
0,73592,IS,522,Advanced Spreadsheet Skills,This is an online course that introduces stud...
1,70426,IS,612,Introduction to Coding,This course provides an introduction to progr...
2,70376,IS,613,Database Management Systems,This course focuses on the theoretical and pr...
3,71721,IS,613,Database Management Systems,This course focuses on the theoretical and pr...
4,72779,IS,613,Database Management Systems,This course focuses on the theoretical and pr...
5,70063,IS,617,Information Systems Principles,This course examines managerial information r...
6,71371,IS,620,IS and Organizational Strategy,This course brings to life the latest busines...
7,70377,IS,623,Info Sys Design & Devlpmt,Not open to students who have taken IS 621. Co...
8,71427,IS,623,Info Sytems Design & Devlpmt,Not open to students who have taken IS 621. Co...
9,72669,IS,628,Research Mthds for User Exp.,"In User Experience, the foundation is researc..."


In [28]:
df_IS['clean_text'] = df_IS['DESCRIPTION:'].apply(clean)
df_IS

,CRN,Subject,Course#,Title,DESCRIPTION:,clean_text
0,73592,IS,522,Advanced Spreadsheet Skills,This is an online course that introduces stud...,online course introduces student feature micr...
1,70426,IS,612,Introduction to Coding,This course provides an introduction to progr...,course provides introduction programming pyth...
2,70376,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
3,71721,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
4,72779,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
5,70063,IS,617,Information Systems Principles,This course examines managerial information r...,course examines managerial information requir...
6,71371,IS,620,IS and Organizational Strategy,This course brings to life the latest busines...,course brings life latest business research c...
7,70377,IS,623,Info Sys Design & Devlpmt,Not open to students who have taken IS 621. Co...,open student taken 621 course description cour...
8,71427,IS,623,Info Sytems Design & Devlpmt,Not open to students who have taken IS 621. Co...,open student taken 621 course description cour...
9,72669,IS,628,Research Mthds for User Exp.,"In User Experience, the foundation is researc...",user experience foundation research required ...


In [29]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector=CountVectorizer()
X_counts = count_vector.fit_transform(df_IS['clean_text'])
vocab=count_vector.get_feature_names()
vs_model_data=pd.DataFrame(X_counts.toarray(), columns=vocab)
vs_model_data

,2016,24,27000,27001,27002,600,621,695q,access,accessing,...,wide,widening,within,without,work,workbook,worksheet,world,written,yet
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Top 20 words

In [30]:
top_words = vs_model_data.sum().sort_values(ascending=False)
top_words.head(20)

data           58
course         49
system         46
management     43
information    29
student        25
security       24
project        19
database       18
include        15
development    15
topic          15
research       15
design         14
business       14
application    14
user           13
rotation       12
software       12
spring         11
dtype: int64

### Longest Description

In [31]:
top_len = df_IS['DESCRIPTION:'].agg(len).sort_values(ascending=False)

In [32]:
# top_len.idxmax()
top_len

27    1758
33    1243
19    1213
31     765
5      740
6      728
0      667
13     660
14     660
15     660
29     618
10     576
9      576
25     554
8      498
7      498
4      494
3      494
2      494
18     453
17     453
1      396
21     396
16     383
23     311
12     271
Name: DESCRIPTION:, dtype: int64

In [33]:
top_index = top_len.idxmax()
top_index

27

In [34]:
df_IS

,CRN,Subject,Course#,Title,DESCRIPTION:,clean_text
0,73592,IS,522,Advanced Spreadsheet Skills,This is an online course that introduces stud...,online course introduces student feature micr...
1,70426,IS,612,Introduction to Coding,This course provides an introduction to progr...,course provides introduction programming pyth...
2,70376,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
3,71721,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
4,72779,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
5,70063,IS,617,Information Systems Principles,This course examines managerial information r...,course examines managerial information requir...
6,71371,IS,620,IS and Organizational Strategy,This course brings to life the latest busines...,course brings life latest business research c...
7,70377,IS,623,Info Sys Design & Devlpmt,Not open to students who have taken IS 621. Co...,open student taken 621 course description cour...
8,71427,IS,623,Info Sytems Design & Devlpmt,Not open to students who have taken IS 621. Co...,open student taken 621 course description cour...
9,72669,IS,628,Research Mthds for User Exp.,"In User Experience, the foundation is researc...",user experience foundation research required ...


In [35]:
df_IS_index = df_IS.index
df_IS_index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17,
            18, 19, 21, 23, 25, 27, 29, 31, 33],
           dtype='int64')

In [36]:
highest_word_course_index = df_IS_index.get_loc(top_index)
highest_word_course_index

22

In [37]:
df_IS.iloc[highest_word_course_index]

CRN                                                         72599
Subject                                                        IS
Course#                                                       669
Title                              Big Data & Information Systems
DESCRIPTION:     Data and analytics are changing the world and...
clean_text       data analytics changing world way making deci...
Name: 27, dtype: object

### Number of words

In [39]:
n_words = vs_model_data.iloc[highest_word_course_index].sum()
n_words

160